Kevin Pham


Research Question
  "Does a home win in an NBA or NFL game lead to a measurable difference in daily restaurant and bar spending in major U.S. Cities?

**Similar Research**


Selling the Game: Estimating the Economic Impact of Professional Sports through Taxable Sales
  
  By Robert A. Baade, Robert Baumann and Victor A. Matheson

Findings:
  - New Facilities and franchises don't reliably boost retail sales
  - Mega-Events have no consistent positive effet
  - Strikes and lockouts (interuptions in a season) have minimal effect




Estimating Local Welfare Generated by an NFL Team under Credible Threat of Relocation

  By Aju J. Fenn and John R. Crooker

Findings:
  - Credible Relocation Threat is credible
      - Peoples willingness to pay doesn't change without perception that franchise might actually leave


Sources of Data
  
  Opportunity insights Economic tracker

   - Affinity Federal Credit Union Credit Card Spending Data


   

  NBA API and Pro Football Reference for Sports Data

In [ ]:
!pip install nba_api
!pip install beautifulsoup4
import pandas as pd
import numpy as np
import requests
from nba_api.stats.endpoints import leaguegamefinder
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from sklearn.linear_model import LassoCV, ElasticNetCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


In [ ]:
#Cleaning Economic Data
Spending_df = pd.read_csv('/content/Affinity - City - Daily.csv')

#Define desired columns and city IDs
spending_cols =['spend_acf','spend_inperson','spend_all']
display_cols = ['cityid', 'year', 'month', 'day']
city_ids = [1, 2, 3, 7, 13, 14, 17, 18, 21, 28, 29, 36, 38, 44]

#Create Spending df
Spending_df[spending_cols] = Spending_df[spending_cols].apply(pd.to_numeric, errors='coerce')
Spending_df = Spending_df.dropna(subset = spending_cols)
Spending_df = Spending_df[Spending_df['cityid'].isin(city_ids)]
Spending_df = Spending_df[display_cols + spending_cols]

#Mapping City IDs to names
ID_to_name = {
    1: 'Los Angeles', 2: 'New York City', 3: 'Chicago', 7: "Dallas", 13: "Detriot", 14: 'Philadelphia', 17: 'Charlotte',
    18: 'Indianapolis', 21: 'San Francisco', 28: 'Denver', 29: 'Washington DC', 36: 'Atlanta', 38: 'Miami', 44: 'New Orleans'}

#Adding city name col
Spending_df['city_name'] = Spending_df['cityid'].map(ID_to_name)


In [ ]:
#Get Sports Data

#Teams needed and mapping cities
nba_teams = [
    'Lakers','Clippers','Knicks','Bulls','Mavericks','Pistons','76ers','Hornets'
    ,'Pacers','Warriors','Nuggets','Wizards','Hawks','Heat','Pelicans']

nfl_teams = [
    'Los Angeles Rams', 'Los Angeles Chargers', 'New York Giants', 'Chicago Bears', 'Dallas Cowboys',
    'Detroit Lions', 'Philadelphia Eagles', 'Carolina Panthers', 'Indianapolis Colts',
    'San Francisco 49ers', 'Denver Broncos', 'Washington Commanders',
    'Atlanta Falcons', 'Miami Dolphins', 'New Orleans Saints']

nba_cities = [
    'Los Angeles','Los Angeles','New York City','Chicago','Dallas','Detroit',
    'Philadelphia','Charlotte','Indianapolis','San Francisco','Denver',
    'Washington DC','Atlanta','Miami','New Orleans']

nba_mapping = dict(zip(nba_teams, nba_cities))

nfl_cities = [
    'Los Angeles','Los Angeles','New York City','Chicago','Dallas','Detroit',
    'Philadelphia','Charlotte','Indianapolis','San Francisco','Denver',
    'Washington DC','Atlanta','Miami','New Orleans']
nfl_mapping = dict(zip(nfl_teams, nfl_cities))

teams_to_city = {}
teams_to_city.update(nba_mapping)
teams_to_city.update(nfl_mapping)


#Scrape NFL data
def nfl_data(start_year, end_year):
    game_list = []
    #Loop for years
    for year in range(start_year, end_year + 1):
        url = f'https://www.pro-football-reference.com/years/{year}/games.htm'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'games'})

        #Read table into a dataframe
        df = pd.read_html(str(table))[0]

        #Filter Columns and clean
        df = df[['Date', 'Winner/tie', 'Loser/tie', 'Unnamed: 7']]
        df['Year'] = year
        #Fix date format
        df['Date'] = pd.to_datetime(df['Date'].str.replace(r".*,\s*","", regex = True) + ' ' + df['Year'].astype(str), errors = 'coerce' )
        df = df[df['Date'].notna()]

        #Map Teams to cities
        df['City'] = df['Winner/tie'].map(teams_to_city).fillna(df['Loser/tie'].map(teams_to_city))
        df['Sport'] = 'NFL'

        #Determine home or away game
        df['home_game'] = df['Unnamed: 7'].apply(lambda x: 0 if x == '@' else 1)

        #Determine win/loss
        df['win'] = df['Winner/tie'].apply(lambda x: 1 if x in nfl_teams else 0)

        game_list.append(df)

    nfl_df = pd.concat(game_list, ignore_index=True)
    return nfl_df

#Function for NBA data
def nba_data(start_year, end_year):
    game_list = []
    #Loop for years
    for year in range(start_year, end_year + 1):
        season = str(year) + '-' + str(year + 1)[-2:]
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
        #Get Schedule
        games = gamefinder.get_data_frames()[0]

        #Filter need NBA teams
        games = games[games['TEAM_NAME'].isin(nba_teams)]
        #Filter Home Games
        games['home_game'] = games['MATCHUP'].apply(lambda x: 1 if 'vs.' not in x else 0)
        #Filter Wins
        games['win'] = games['WL'].apply(lambda x: 1 if x == 'W' else 0)
        games['city_name'] = games['TEAM_NAME'].map(teams_to_city)
        games['sport'] = 'NBA'
        game_list.append(games[['TEAM_NAME','city_name','GAME_DATE','home_game','win','sport']])
    nba_df = pd.concat(game_list, ignore_index=True)
    nba_df.rename(columns={'TEAM_NAME': 'Team', 'GAME_DATE': 'Date'}, inplace=True)
    return nba_df

#get data for both NFL and NBA
nfl_games = nfl_data(2020, 2024)
nba_games = nba_data(2020, 2024)

#Keep relevant nfl cities
nfl_games = nfl_games[nfl_games['City'].notna()]

#Normalize column names
nfl_games = (nfl_games.rename(columns={'Winner/tie':'Team','City': 'city_name'})[['Date','Team','city_name','home_game','win','Sport']])
nba_games = (nba_games.rename(columns={'TEAM_NAME':'Team','city_name': 'city_name'})[['Date','Team','city_name','home_game','win','sport']])

#Remove time from output
nfl_games['Date'] = nfl_games['Date'].dt.floor('d')
nba_games['Date'] = pd.to_datetime(nba_games['Date']).dt.floor('d')


#Combine
sports_games = pd.concat([nfl_games, nba_games], ignore_index=True)


In [ ]:
#Merging Spending and Sports data

#Formatting Spending DF
Spending_df['Date'] = pd.to_datetime(Spending_df[['year','month','day']]).dt.floor('d')
Spending_df = Spending_df.rename(columns = {'city':'city_name'})

#Merging
merged_df = pd.merge( Spending_df,sports_games, on = ['city_name','Date'], how = 'left')
merged_df[['home_game','win']] = merged_df[['home_game','win']].fillna(0).astype(int)
merged_df['dow'] = merged_df['Date'].dt.day_name() # Day of Week
merged_df['month'] = merged_df['Date'].dt.month # Month

X = merged_df[['home_game','win']].values
y = merged_df['spend_acf'].values

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

#Baseline OLS
X0 = sm.add_constant(merged_df['home_game'])
print('\nBaseline OLS:')
print(sm.OLS(merged_df['spend_acf'], X0).fit().summary())

#Create Dummies
dow_dummy = pd.get_dummies(merged_df['dow'], prefix = 'dow', drop_first = True)
month_dummy = pd.get_dummies(merged_df['month'], prefix = 'month', drop_first = True)
city_dummy = pd.get_dummies(merged_df['city_name'], prefix = 'city', drop_first = True)

Fixed_effects = pd.concat([merged_df[['home_game','win']],dow_dummy,month_dummy,city_dummy], axis = 1)
Fixed_effects_const = sm.add_constant(Fixed_effects).astype(float)

#Cluster OLS w/ City dummies
y = merged_df['spend_acf'].values

ols_clustered = sm.OLS(y, Fixed_effects_const).fit(cov_type = 'cluster',cov_kwds={'groups': merged_df['city_name']})
print('\nClustered OLS with DOW, Month, and City dummies:')
print(ols_clustered.summary())

#PCA
#Create df of dummies
dummies = pd.concat([dow_dummy, month_dummy, city_dummy], axis = 1)
#Standardze dummies
dummy_std = StandardScaler().fit_transform(dummies)

pca = PCA(n_components = 10).fit(dummy_std)
pca_result = pca.transform(dummy_std)
#Combine home game, wins and components
pca_df = np.column_stack ([merged_df[['home_game','win']].values,pca_result])
pca_df = sm.add_constant(pca_df)

model = sm.OLS(y, pca_df).fit()
print('\nPCA Results:')
print(model.summary())

#Alphas for regression
alphas_reg = np.logspace(-6,2,50)

#Ridge
ridge_std = StandardScaler().fit_transform(Fixed_effects)
ridge = RidgeCV(alphas = alphas_reg, cv = 5).fit(ridge_std, y)
print("\nRidge alpha:", ridge.alpha_)


#Elastic Net
elastic = ElasticNetCV(alphas = alphas_reg, l1_ratio=[0.1,0.5,0.9], cv = 5,max_iter = 10000).fit(X_std, y)
print("\nElasticNet alpha:", elastic.alpha_)

#LASSO
lasso_std = StandardScaler().fit_transform(Fixed_effects.values)
lasso = LassoCV(alphas = alphas_reg, cv = 5, max_iter = 10000).fit(lasso_std, y)
print("\nNonzero LASSO coefs:", [f for f, c in zip(Fixed_effects.columns, lasso.coef_) if c != 0])




Baseline OLS:
                            OLS Regression Results                            
Dep. Variable:              spend_acf   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     8.056
Date:                Sun, 11 May 2025   Prob (F-statistic):            0.00454
Time:                        00:55:06   Log-Likelihood:                 268.73
No. Observations:               10813   AIC:                            -533.5
Df Residuals:                   10811   BIC:                            -518.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1332      0.002    -

Takeaways
 - Baseline OLS shows that home games create a boost in spending
    - R^2 value shows that this model isn't significant and home games don't really explain much
 - RidgeCV and ElasticNetCV show large penalities, shrinking all sports related factors significantly

 - PCR shows that home games or wins don't have a great effect when other facotrs are taken into consideration

 - Home games or wins and losses dont have much of an effect on restaurant spending overall.